In [1]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool

In [2]:
from typing import Optional, Type
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

In [1]:
%env OPENAI_API_KEY=<>

env: OPENAI_API_KEY=<>


In [4]:
import coffifilter

In [14]:
coffifilter.init(
    redis_host="",
    redis_port=11552,
    redis_db=0,
    redis_password=""
)

In [15]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

In [16]:
class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput
    return_direct: bool = False

    def _run(
        self, a: int, b: int, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return a * b

    async def _arun(
        self,
        a: int,
        b: int,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("Calculator does not support async")

In [17]:
calculator_tool = coffifilter.wrap_langchain_tool(CustomCalculatorTool())

In [18]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")

In [19]:
tools = [calculator_tool]

In [20]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [21]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [22]:
agent_executor.invoke({"input": "what is seven times 49"})



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `{'a': 7, 'b': 49}`


343Seven times 49 is equal to 343.

> Finished chain.


{'input': 'what is seven times 49',
 'output': 'Seven times 49 is equal to 343.'}